# KoruDelta Tutorial

**The Causal Database for AI Agents**

This notebook demonstrates KoruDelta's unique capabilities:
- **Causal Storage**: Every change tracked with provenance
- **Vector Search**: Semantic memory with embeddings
- **Time Travel**: Query historical state
- **Natural Lifecycle**: Hot→Warm→Cold→Deep memory tiers

## 1. Setup

In [ ]:
import asyncio
import koru_delta

print(f"KoruDelta version: {koru_delta.__version__}")

## 2. Basic Operations

Store and retrieve data with automatic versioning.

In [ ]:
async def basic_demo():
    async with koru_delta.Database() as db:
        # Store data
        await db.put("users", "alice", {
            "name": "Alice",
            "email": "alice@example.com",
            "role": "developer"
        })
        
        # Retrieve data
        user = await db.get("users", "alice")
        print(f"User: {user['name']} ({user['email']})")
        
        # Check existence
        exists = await db.contains("users", "alice")
        print(f"Key exists: {exists}")
        
        # List keys
        keys = await db.list_keys("users")
        print(f"Keys: {keys}")
        
        # Get stats
        stats = await db.stats()
        print(f"Stats: {stats}")

await basic_demo()

## 3. Vector Embeddings & Semantic Search

Store vectors and search by semantic similarity.

In [ ]:
async def vector_demo():
    async with koru_delta.Database() as db:
        # Store vectors (simulating embeddings)
        await db.embed("documents", "doc1", 
                      embedding=[0.9, 0.1, 0.2, 0.8, 0.3],
                      model="text-embedding-3-small",
                      metadata={"title": "Introduction to AI"})
        
        await db.embed("documents", "doc2",
                      embedding=[0.85, 0.15, 0.25, 0.75, 0.35],
                      model="text-embedding-3-small",
                      metadata={"title": "Machine Learning Basics"})
        
        await db.embed("documents", "doc3",
                      embedding=[0.1, 0.9, 0.8, 0.1, 0.9],
                      model="text-embedding-3-small",
                      metadata={"title": "Cooking Recipes"})
        
        # Search for similar vectors
        query = [0.88, 0.12, 0.22, 0.82, 0.32]  # Similar to doc1/doc2
        results = await db.similar("documents", query=query, top_k=3)
        
        print("Similar documents:")
        for r in results:
            print(f"  {r['key']}: score={r['score']:.3f}")

await vector_demo()

## 4. Time Travel

Query historical state - see what data looked like at any point in time.

In [ ]:
async def time_travel_demo():
    async with koru_delta.Database() as db:
        from datetime import datetime
        
        # Store initial value
        await db.put("config", "api-version", {"version": "1.0", "status": "stable"})
        print("Stored: v1.0")
        
        # Update value
        await db.put("config", "api-version", {"version": "2.0", "status": "beta"})
        print("Updated: v2.0")
        
        # Get current value
        current = await db.get("config", "api-version")
        print(f"Current: {current}")
        
        # Get history
        history = await db.history("config", "api-version")
        print(f"\nHistory ({len(history)} versions):")
        for i, entry in enumerate(history, 1):
            val = entry.get("value", {})
            print(f"  {i}. v{val.get('version', 'unknown')} - {val.get('status', 'unknown')}")

await time_travel_demo()

## 5. AI Agent Memory

Use the agent memory interface for episodic, semantic, and procedural memory.

In [ ]:
async def agent_memory_demo():
    async with koru_delta.Database() as db:
        # Create agent memory
        memory = db.agent_memory("assistant-001")
        
        # Store episodic memory (events)
        await memory.episodes.remember(
            "User asked about Python integration",
            importance=0.8,
            tags=["python", "integration", "question"]
        )
        print("✓ Stored episode")
        
        # Store semantic memory (facts)
        await memory.facts.learn(
            "user_name",
            "User's name is Alice",
            tags=["personal", "identity"]
        )
        print("✓ Learned fact")
        
        # Store procedural memory (how-to)
        await memory.procedures.learn(
            "explain_vector_search",
            steps=[
                "1. Explain embedding concept",
                "2. Show similarity calculation",
                "3. Demonstrate semantic search"
            ],
            success_rate=0.9
        )
        print("✓ Learned procedure")
        
        # Recall memories
        results = await memory.recall("Python", limit=5)
        print(f"\nRecalled {len(results)} memories about 'Python':")
        for r in results:
            print(f"  [{r.relevance:.2f}] {r.content[:50]}...")
        
        # Get stats
        stats = await memory.stats()
        print(f"\nMemory stats: {stats}")

await agent_memory_demo()

## 6. Configuration

Configure persistence and performance settings.

In [ ]:
# Example with persistence
from koru_delta import Config

config = Config(
    path="~/.korudelta/demo",  # Persist to disk
    max_memory_mb=1024,         # Limit memory
    enable_wal=True             # Write-ahead log
)

async def persistent_demo():
    async with koru_delta.Database(config) as db:
        await db.put("persistent", "key", {"data": "survives restart"})
        value = await db.get("persistent", "key")
        print(f"Stored: {value}")
        print("Data will persist across restarts!")

await persistent_demo()

## Summary

KoruDelta provides:

1. **Causal Storage**: Every change tracked
2. **Vector Search**: Semantic similarity
3. **Time Travel**: Historical queries
4. **Agent Memory**: Episodic, semantic, procedural
5. **Edge-Ready**: 8MB binary, runs anywhere

**Next Steps:**
- Try the full examples: `examples/02_ai_agent.py`
- Read the docs: https://docs.korudelta.dev
- Join the community: https://github.com/swyrknt/koru-delta